In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [4]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
import data_process, path
import data_generator
from data_functions import make_date_to_MYR, make_MYR_to_date
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from path import col_seq_can, col_seq_can_all
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

In [5]:
%config Completer.use_jedi = False

### Load Pmpm and IDs

In [6]:
# import boto3
# s3 = boto3.client('s3')
# bucket = 'cdphp-s3-us-e-p-pond'
# path_to_save = '../../../data/PMPM.p'
# s3_path = 'rpi/nneehal/DATA/pmpm_202108_processed_ready.p'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [7]:
# path_to_save = '../../../data/ids.csv'
# s3_path = 'rpi/blind_matching_study/input_data/RPI Test Cohort.csv'
# success = s3.download_file(bucket, s3_path, path_to_save)

In [8]:
%%time
ids_csv = '../../../data/ids.csv'
pmpm_p = '../../../data/PMPM.p'

treated_ids = pd.read_csv(ids_csv)
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)


CPU times: user 8.84 s, sys: 10.8 s, total: 19.6 s
Wall time: 19.6 s


In [9]:
treated_ids.shape

### Drop Unecessary data points

In [10]:
pmpm_backup = pmpm.copy()

In [11]:
pmpm_backup.MYR.min(), pmpm_backup.MYR.max()

(201701, 202108)

In [12]:
#pmpm_backup2 = pmpm_backup[pmpm_backup.MYR >= 202003].reset_index(drop = True)

In [13]:
pmpm_backup.shape

In [14]:
pmpm = pmpm[pmpm.MYR >= 202103].reset_index(drop = True)

In [15]:
pmpm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  202103        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  202104        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  202105        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  202106 

In [145]:
pmpm.tail(50)

PERS_ID     MYR  MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT   MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  \
1170982  5079478  202108         1               22          0          Other      0.00      0.00     0.00          0.00               0.00    0.00       0.00                 0.00          0.00       0.00           0.00       0.00        0.00    0.00              0.00         0.00               0.00               0.00                  0.00     0.00                0.00           0.00                     0.00               0.00         0.00                0          0                   0.00         0.00             0.00         0.00               0.00              0.00            0.00                  0.00                    0.00                  0.00                    0.00                        0.00            0.00            0.00              0.00             0.00               0.00             0.00            0.00          0.00         0.00            0.00          0.00      0.00            0.00         0.00                0.00                   0.00                   0.00   
1170983  5079482  202108         0               20          0     Commercial      0.00      0.00     0.00          0.00               0.00    0.00       0.00                 0.00          0.00       0.00           0.00       0.00        0.00    0.00              0.00         0.00               0.00               0.00                  0.00     0.00                0.00           0.00                     0.00               0.00         0.00                0          0                   0.00         0.00             0.00         0.00               0.00              0.00            0.00                  0.00                    0.00                  0.00                    0.00                        0.00            0.00            0.00              0.00             0.00               0.00             0.00            0.00          0.00         0.00            0.00          0.00      0.00            0.00         0.00                0.00                   0.00                   0.00   
1170984  5079506  202108         0               18          0          Other      0.00      0.00     0.00          0.00               0.00    0.00       0.00                 0.00          0.00       0.00           0.00       0.00        0.00    0.00              0.00         0.00               0.00               0.00                  0.00     0.00                0.00           0.00                     0.00               0.00         0.00                0          0                   0.00         0.00             0.00         0.00               0.00              0.00            0.00                  0.00                    0.00                  0.00                    0.00                        0.00            0.00            0.00              0.00             0.00               0.00             0.00            0.00          0.00         0.00            0.00          0.00      0.00            0.00         0.00                0.00                   0.00                   0.00   
1170985  5079553  202108         

In [16]:
pmpm.MBR_GNDR.replace(to_replace = {'F':0, 'M': 1}, inplace = True)
# pmpm.MEDICAID_LOB.unique()

### Include Line of Bussiness Feature 1 Not medicaid 0 medicaid

In [17]:
# medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
# pmpm['LOB'] = 1
# pmpm.loc[medicaid_mask, 'LOB'] = 0

### Separate Treated Controls

In [18]:
treated_ids.iloc[0]

PERS_ID    2778075
Name: 0, dtype: int64

In [19]:
pmpm.PERS_ID = pmpm.PERS_ID.astype(int)

In [20]:
%%time
id_filter = pmpm.PERS_ID.isin(list(treated_ids.values[:,0]))
treated, controls = pmpm[id_filter].reset_index(drop = True), pmpm[~id_filter].reset_index(drop = True)

CPU times: user 433 ms, sys: 349 ms, total: 782 ms
Wall time: 781 ms


In [21]:
treated = treated[treated.MYR >= 202108].reset_index(drop = True)

# Take average of treated and controls

### Generate Treated

In [22]:
treated2 = data_process.get_average_record(df=treated, pmpm=pmpm_backup.copy())

PERS_ID
First date
One year before
history from original pmpm
record size calculated
record filtered
returning from f1...
f1 done


100%|██████████| 97/97 [00:00<00:00, 32707.41it/s]

f2 done
stacking done
returning..


In [23]:
treated2.ALLOW_AMT = treated2.ALLOW_AMT.apply(data_process.log_)
treated2.ALLOW_IP = treated2.ALLOW_IP.apply(data_process.log_)
treated2.ALLOW_ER = treated2.ALLOW_ER.apply(data_process.log_)

In [24]:
#treated2.to_pickle("jeremy_avg_treated.p")

### Generate Controls 

In [25]:
controls2 = data_process.get_average_controls_jeremy(controls.copy(), pmpm_backup.copy())

[-12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
(158288, 92)
(158288, 77)
[-13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2]
(162237, 92)
(320525, 77)
[-14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3]
(165840, 92)
(486365, 77)
[-15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4]
(168384, 92)
(654749, 77)
[-16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5]
(169947, 92)
(824696, 77)
[-17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6]
(169841, 92)
(994537, 77)


In [26]:
controls2.ALLOW_AMT = controls2.ALLOW_AMT.apply(data_process.log_)
controls2.ALLOW_IP = controls2.ALLOW_IP.apply(data_process.log_)
controls2.ALLOW_ER = controls2.ALLOW_ER.apply(data_process.log_)

In [27]:
#controls2.to_pickle("jeremy_avg_controls.p")

### Save in S3 for reuse

In [28]:
# !aws s3 cp jeremy_avg_treated.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_treated.p --acl bucket-owner-full-control
# !aws s3 cp jeremy_avg_controls.p s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Blind_Study_New/avg_controls.p --acl bucket-owner-full-control

In [29]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/get_matches1_8.ipynb s3_get_matches1_8.ipynb --acl bucket-owner-full-control

In [34]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/mapping.xlsx mapping.xlsx --acl bucket-owner-full-control

download: s3://cdphp-s3-us-e-p-pond/rpi/blind_matching_study/results/mapping.xlsx to ./mapping.xlsx


## Match begin

In [30]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE', 'ACUTE2']

In [44]:
def get_data(data_struct, cont):
    cont = cont.copy()
    cont.PERS_ID = cont.PERS_ID.astype(str)
    keys = list(data_struct.keys())
    dat1 = pd.DataFrame()
    for key in keys:
        dat1 = pd.concat([dat1, data_struct[key]], ignore_index = True)
    dat2 = dat1[['PERS_ID', 'MYR']]
    dat = pd.merge(dat2, cont, on = ['PERS_ID', 'MYR']).reset_index(drop = True)
    return dat

### General Configurations for All Matching Scenaria

In [32]:
#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
hier_cols = None #['CC_DX_CNT']
hier_range = [1]
exact_match_dict = {'ACUTE':None, 'ACUTE2':None}
match_on = [ 'ALLOW_AMT', 'CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT']
            


In [33]:
cols_to_keep = ["PERS_ID", "MYR", 'AGE_AT_MIDMONTH', 'MBR_GNDR']

## Load Model

In [35]:
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras import models, layers
import tensorflow.keras.backend as K

model = models.load_model("../../../temp/TF_Latent_Model_m6_Canary_stacked", compile=False)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [36]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [37]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated2[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
cntrls = data_process.transform_features_with_NN(model, controls2[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

CPU times: user 10 s, sys: 2.76 s, total: 12.8 s
Wall time: 7.37 s


## Scenario 1 Rigorous

In [45]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat1 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:25,  3.90it/s]


CPU times: user 1min 33s, sys: 1min 16s, total: 2min 49s
Wall time: 27.7 s


In [46]:
dat1.shape

## Scenario 2 Non Rigorous

In [47]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat2 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:25,  3.80it/s]


CPU times: user 1min 40s, sys: 1min 19s, total: 2min 59s
Wall time: 28.6 s


In [48]:
dat2.shape

### Scenario 3 Rigorous 6 months

In [49]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat3 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:24,  3.96it/s]


CPU times: user 1min 38s, sys: 1min 19s, total: 2min 57s
Wall time: 27.5 s


In [50]:
dat3.MYR.min(), dat3.MYR.max()

In [51]:
dat3.shape

## Scenario 4 Non Rigorous 6 Months

In [52]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat4 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


0it [00:00, ?it/s]

After filtering control ids are:177813 before they were:177813


98it [00:25,  3.91it/s]


CPU times: user 1min 39s, sys: 1min 20s, total: 3min
Wall time: 27.9 s


In [53]:
dat4.shape

## Scenario 5 Medicaid Rigorous 

In [54]:
controlsM = controls[controls.LOB == 0].reset_index(drop = True)

In [55]:
cntrlsM = data_process.transform_features_with_NN(model, controlsM[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

In [59]:
cntrlsM.PERS_ID = cntrlsM.PERS_ID.astype(str)

In [60]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat5 = get_data(rig_st, controls)

2it [00:00, 16.35it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312


98it [00:06, 16.22it/s]


CPU times: user 52 s, sys: 47.1 s, total: 1min 39s
Wall time: 8.29 s


In [61]:
dat5.shape

## Scenario 6 Medicaid Non Rigorous

In [62]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat6 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


1it [00:00,  9.55it/s]

After filtering control ids are:53312 before they were:53312


98it [00:06, 15.02it/s]


CPU times: user 52.3 s, sys: 48.8 s, total: 1min 41s
Wall time: 8.81 s


In [63]:
dat6.shape

### Scenario 7 Medicaid Rigorous 6 months

In [64]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat7 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func


2it [00:00, 18.01it/s]

After filtering control ids are:53312 before they were:53312


98it [00:06, 16.21it/s]


CPU times: user 52 s, sys: 47.1 s, total: 1min 39s
Wall time: 8.3 s


In [65]:
dat7.MYR.min(), dat7.MYR.max()

(202103, 202108)

In [66]:
dat7.shape

## Scenario 8 Medicaid Non Rigorous 6 Months

In [67]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat8 = get_data(rig_st, controls)

0it [00:00, ?it/s]

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312


98it [00:06, 14.97it/s]


CPU times: user 51.1 s, sys: 47.1 s, total: 1min 38s
Wall time: 8.84 s


In [68]:
dat8.shape

# Save Stats

In [94]:
from utils import statistics_first_month
import scipy.stats as sts

In [95]:
columns_to_match = treated.columns.tolist()

In [96]:
%%time
tr_desc0, Ntr, treated_first_month = statistics_first_month(treated, columns_to_match, return_array = True)
tr_desc0 = tr_desc0.T

con1, N1, controls_first_month = statistics_first_month(dat1, columns_to_match, return_array = True)
con1 = con1.T

con2, N2, controls_first_month = statistics_first_month(dat2, columns_to_match, return_array = True)
con2 = con2.T

con3, N3, controls_first_month = statistics_first_month(dat3, columns_to_match, return_array = True)
con3 = con3.T

con4, N4, controls_first_month = statistics_first_month(dat4, columns_to_match, return_array = True)
con4 = con4.T

con5, N5, controls_first_month = statistics_first_month(dat5, columns_to_match, return_array = True)
con5 = con5.T

con6, N6, controls_first_month = statistics_first_month(dat6, columns_to_match, return_array = True)
con6 = con6.T

con7, N7, controls_first_month = statistics_first_month(dat7, columns_to_match, return_array = True)
con7 = con7.T

con8, N8, controls_first_month = statistics_first_month(dat8, columns_to_match, return_array = True)
con8 = con8.T

un_con_all, N9, controls_first_month =  statistics_first_month(controls, columns_to_match, return_array = True)
un_con_all = un_con_all.T

un_con_med, N10, controls_first_month =  statistics_first_month(controlsM, columns_to_match, return_array = True)
un_con_med = un_con_med.T



CPU times: user 1min 48s, sys: 1.69 s, total: 1min 50s
Wall time: 1min 50s


In [97]:
tr_desc0

mean          std  count
MYR                         202108.000000     0.000000  111.0
MBR_GNDR                         0.342342     0.476645  111.0
AGE_AT_MIDMONTH                 37.873874    11.873666  111.0
ALLOW_AMT                      709.126126  1425.406093  111.0
ALLOW_ER                        22.187207   110.864186  111.0
ALLOW_IP                       279.282523  1205.828308  111.0
CC_ADHD                          0.225225     0.419625  111.0
CC_ALZHEIMER                     0.009009     0.094916  111.0
CC_ALZHEIMER_PLUS                0.036036     0.187225  111.0
CC_AMI                           0.018018     0.133620  111.0
CC_ANEMIA                        0.162162     0.370271  111.0
CC_ANXIETY_DISORDER              0.900901     0.300150  111.0
CC_ARTHRITIS                     0.144144     0.352829  111.0
CC_ASTHMA                        0.207207     0.407143  111.0
CC_ATRIAL_FIB                    0.036036     0.187225  111.0
CC_AUTISM                        0.036036     0.187225  111.0
CC_BIPOLAR                       0.414414     0.494855  111.0
CC_BPH                           0.009009     0.094916  111.0
CC_BREAST_CANCER                 0.009009     0.094916  111.0
CC_CATARACT                      0.000000     0.000000  111.0
CC_CEREBRAL_PALSY                0.000000     0.000000  111.0
CC_CHRONIC_KIDNEY                0.270270     0.446113  111.0
CC_COLORECTAL_CANCER             0.009009     0.094916  111.0
CC_COPD                          0.135135     0.343418  111.0
CC_CYSTIC_FIBROSIS               0.000000     0.000000  111.0
CC_DEPRESSION                    0.801802     0.400450  111.0
CC_DEPRESSIVE_DISORDERS          0.108108     0.311925  111.0
CC_DEVELOP_DELAYS                0.009009     0.094916  111.0
CC_DIABETES                      0.189189     0.393435  111.0
CC_DISAB_DX_CNT                  0.198198     0.536314  111.0
CC_DX_CNT                        7.063063     3.739906  111.0
CC_ENDOMETRIAL_CANCER            0.009009     0.094916  111.0
CC_EPILEPSY                      0.063063     0.244179  111.0
CC_FIBROMYALGIA                  0.333333     0.473542  111.0
CC_GLAUCOMA                      0.027027     0.162898  111.0
CC_HEARING_IMPAIR                0.000000     0.000000  111.0
CC_HEART_FAILURE                 0.108108     0.311925  111.0
CC_HEPATITIS_A                   0.000000     0.000000  111.0
CC_HEPATITIS_B_ACUTE             0.000000     0.000000  111.0
CC_HEPATITIS_B_CHRONIC           0.000000     0.000000  111.0
CC_HEPATITIS_C_ACUTE             0.000000     0.000000  111.0
CC_HEPATITIS_C_CHRONIC           0.018018     0.133620  111.0
CC_HEPATITIS_C_UNSPECIFIED       0.027027     0.162898  111.0
CC_HEPATITIS_D                   0.000000     0.000000  111.0
CC_HEPATITIS_E                   0.000000     0.000000  111.0
CC_HEPATITIS_GEN                 0.036036     0.187225  111.0
CC_HIP_FRACTURE                  0.000000     0.000000  111.0
CC_HYPERLIPIDEMIA                0.189189     0.393435  111.0
CC_HYPERTENSION                  0.297297     0.459141  111.0
CC_HYPOTHYROID                   0.072072     0.259780  111.0
CC_INT_DISAB                     0.018018     0.133620  111.0
CC_ISCHEMIC                      0.108108     0.311925  111.0
CC_LEARN_DISAB                   0.018018     0.133620  111.0
CC_LEUKEMIAS                     0.000000     0.000000  111.0
CC_LIVER                         0.162162     0.370271  111.0
CC_LUNG_CANCER                   0.000000     0.000000  111.0
CC_MIGRAINE                      0.099099     0.300150  111.0
CC_MOBILITY_IMPAIR               0.027027     0.162898  111.0
CC_MULTIPLE_SCLEROSIS            0.000000     0.000000  111.0
CC_MUSCULAR_DYSTROPHY            0.000000     0.000000  111.0
CC_OBESITY                       0.324324     0.470245  111.0
CC_OSTEOPOROSIS                  0.009009     0.094916  111.0
CC_PERIPHERAL                    0.009009     0.094916  111.0
CC_PERSONALITY                   0.216216     0.413530  111.0
CC_PRES

In [98]:
alli = pd.concat((tr_desc0[['mean','std']], un_con_all[['mean','std']], con1[['mean','std']], con2[['mean','std']], con3[['mean','std']], 
                  con4[['mean','std']], un_con_med[['mean','std']], con5[['mean','std']], con6[['mean','std']], con7[['mean','std']], con8[['mean','std']]),
                 axis = 1, keys = ['Treated', 'Unmatched Controls', 'Rig', 'NonRig', 'Rig6', 'NonRig6', 'Unmatched Med_Controls', 
                                   'Rig-Med', 'NonRig-Med', 'Rig-Med6','NonRig-Med6'])

In [99]:
pd.options.display.float_format = '{:.2f}'.format

In [100]:
alli[2:]

Treated         Unmatched Controls              Rig         NonRig            Rig6         NonRig6         Unmatched Med_Controls           Rig-Med       NonRig-Med       Rig-Med6       NonRig-Med6      
                              mean     std               mean       std   mean     std   mean     std    mean     std    mean     std                   mean       std    mean   std       mean   std     mean   std        mean   std
AGE_AT_MIDMONTH              37.87   11.87              48.09     20.25  38.70   11.92  38.91   11.95   38.75   11.86   39.09   12.07                  34.04     16.91   38.08 11.49      37.85 11.34    38.10 11.79       37.78 11.63
ALLOW_AMT                   709.13 1425.41            1030.41   5321.78 361.12 1157.89 319.83 1299.27 1819.56 5404.15 1379.04 5175.66                 822.60   2931.48   20.39 29.03      24.91 36.18    16.52 17.87       25.00 28.52
ALLOW_ER                     22.19  110.86              20.08    192.30   7.96   75.79   6.03   69.60   25.95  145.67   31.12  226.28                  23.18    140.56    0.00  0.00       0.19  4.74     0.00  0.00        0.00  0.00
ALLOW_IP                    279.28 1205.83             249.13   4264.12   0.00    0.00   0.00    0.00  684.05 4358.33  406.52 4097.60                 192.86   2071.91    0.00  0.00       0.00  0.00     0.00  0.00        0.00  0.00
CC_ADHD                       0.23    0.42               0.04      0.20   0.11    0.31   0.07    0.26    0.10    0.30    0.09    0.29                   0.09      0.28    0.08  0.27       0.09  0.28     0.07  0.25        0.06  0.23
CC_ALZHEIMER                  0.01    0.09               0.01      0.07   0.00    0.00   0.00    0.00    0.00    0.00    0.00    0.00                   0.00      0.03    0.00  0.00       0.00  0.02     0.00  0.00        0.00  0.03
CC_ALZHEIMER_PLUS             0.04    0.19               0.02      0.13   0.01    0.09   0.00    0.07    0.00    0.05    0.01    0.08                   0.01      0.08    0.01  0.08       0.01  0.09     0.01  0.08        0.01  0.08
CC_AMI                        0.02    0.13               0.00      0.06   0.00    0.05   0.00    0.02    0.00    0.05    0.00    0.05                   0.00      0.05    0.00  0.06       0.01  0.07     0.00  0.07        0.00  0.05
CC_ANEMIA                     0.16    0.37               0.07      0.26   0.07    0.26   0.07    0.25    0.14    0.35    0.11    0.31                   0.07      0.26    0.08  0.27       0.08  0.27     0.06  0.23        0.08  0.27
CC_ANXIETY_DISORDER           0.90    0.30               0.22      0.42   0.43    0.50   0.33    0.47    0.56    0.50    0.47    0.50                   0.29      0.46    0.41  0.49       0.38  0.49     0.39  0.49        0.37  0.48
CC_ARTHRITIS                  0.14    0.35               0.14      0.34   0.14    0.35   0.10    0.30    0.19    0.40    0.16    0.36                   0.08      0.27    0.10  0.30       0.09  0.28     0.08  0.27        0.09  0.28
CC_ASTHMA                     0.21    0.41               0.06      0.23   0.11    0.31   0.08    0.27    0.17    0.37    0.12    0.33                   0.09      0.28    0.10  0.30       0.08  0.28     0.09  0.28        0.09  0.28
CC_ATRIAL_FIB                 0.04    0.19               0.03      0.17   0.01    0.10   0.00    0.06    0.01    0.12    0.01    0.10                   0.01      0.09    0.00  0.05       0.01  0.08     0.00  0.05        0.01  0.07
CC_AUTISM                     0.04    0.19               0.01      0.08   0.02    0.13   0.01    0.11    0.02    0.13    0.02    0.13                   0.02      0.13    0.01  0.12       0.01  0.11     0.01  0.08        0.01  0.09
CC_BIPOLAR                    0.41    0.49               0.04      0.19   0.08    0.28   0.06    0.23    0.12    0.33    0.09    0.28                   0.08      0.27    0.14  0.34       0.11  0.31     0.09  0.28        0.09  0.29
CC_BPH                        0.01    0.09               0.03      0.16   0.00    0.06   0.00    0.0

In [102]:
Path('./results/stats_mean_std').mkdir(exist_ok = True)
alli[2:].to_csv("./results/stats_mean_std/seq_1_8_mean_std.csv", header=True, index=True)

## Latent Stats

In [103]:
dat_lists = [dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8]
stat = []
for data in dat_lists:
    dat_lat = data_process.transform_features_with_NN(model, data[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
    stat_a, N, _ = statistics_first_month(dat_lat, latent_cols, return_array = True)
    stat_a = stat_a.T
    stat.append(stat_a)   

In [104]:
tr_stats, N, _ =  statistics_first_month(tr2, latent_cols, return_array = True)
tr_stats = tr_stats.T

In [105]:
alli_latent = pd.concat((tr_stats[['mean','std']], stat[0][['mean','std']], stat[1][['mean','std']], stat[2][['mean','std']], stat[3][['mean','std']], 
                  stat[4][['mean','std']], stat[5][['mean','std']], stat[6][['mean','std']], stat[7][['mean','std']]),
                 axis = 1, keys = ['Treated', 'seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6', 'seq7','seq8'])

In [106]:
alli_latent

Treated        seq1         seq2         seq3          seq4         seq5       seq6       seq7       seq8     
       mean  std   mean   std   mean   std   mean    std   mean    std  mean  std  mean  std  mean  std  mean  std
x1     0.60 1.40  -1.35  6.01  -1.12  6.63  -9.92  34.42  -7.25  32.76  0.60 1.39  0.62 1.38  0.59 1.40  0.59 1.40
x2    -0.07 0.13   0.03  0.28   0.02  0.31   0.41   1.47   0.30   1.40 -0.07 0.13 -0.08 0.13 -0.07 0.13 -0.07 0.14
x3     1.65 4.34  10.36 29.43   9.36 32.66  52.96 174.26  39.47 165.86  1.62 4.32  1.57 4.30  1.68 4.35  1.67 4.34
x4    -0.03 0.05  -0.10  0.23  -0.09  0.26  -0.43   1.35  -0.33   1.28 -0.03 0.05 -0.02 0.05 -0.03 0.05 -0.03 0.05
x5    -0.47 0.37  -0.42  0.43  -0.43  0.46  -0.78   1.87  -0.65   1.79 -0.47 0.37 -0.48 0.37 -0.47 0.37 -0.47 0.37
x6    -0.05 2.48   4.35 14.43   3.84 15.99  25.18  84.85  18.61  80.75 -0.06 2.47 -0.09 2.45 -0.03 2.48 -0.04 2.48
x7    -1.45 3.74  -8.79 24.62  -7.95 27.32 -44.38 145.33 -33.12 138.32 -1.43 3.72 -1.39 3.70 -1.48 3.74 -1.47 3.74
x8    -2.39 5.26 -13.08 36.20 -11.85 40.18 -65.51 214.64 -48.89 204.29 -2.36 5.23 -2.29 5.21 -2.43 5.27 -2.42 5.27
x9     0.25 0.05   0.27  0.17   0.27  0.19   0.50   0.98   0.43   0.93  0.25 0.05  0.25 0.05  0.25 0.05  0.25 0.05
x10    0.16 0.06   0.33  0.64   0.31  0.71   1.25   3.82   0.96   3.64  0.16 0.06  0.16 0.06  0.16 0.06  0.16 0.06
x11    0.81 0.63   0.38  1.05   0.44  1.11  -0.88   4.68  -0.51   4.45  0.81 0.62  0.82 0.62  0.81 0.63  0.81 0.63
x12   -0.02 0.07  -0.17  0.51  -0.15  0.57  -0.92   3.05  -0.68   2.90 -0.02 0.07 -0.02 0.07 -0.02 0.07 -0.02 0.07
x13    0.04 0.02   0.01  0.11   0.01  0.12  -0.16   0.67  -0.10   0.63  0.04 0.02  0.04 0.02  0.04 0.02  0.04 0.02
x14   -0.41 2.66  -5.30 16.13  -4.74 17.89 -28.59  94.86 -21.24  90.28 -0.40 2.64 -0.37 2.63 -0.43 2.67 -0.43 2.66
x15    0.13 0.02   0.10  0.12   0.10  0.13  -0.08   0.71  -0.02   0.67  0.13 0.02  0.13 0.02  0.13 0.02  0.13 0.02
x16   -0.30 0.17  -0.35  0.41  -0.35  0.46  -0.90   2.43  -0.71   2.32 -0.31 0.17 -0.31 0.17 -0.31 0.17 -0.31 0.17

In [107]:
Path('./results/latent_stats_mean_std').mkdir(exist_ok = True)
alli_latent.to_csv("./results/latent_stats_mean_std/seq_1_8_latent_mean_std.csv", index=True, header=True)

## Save latent features

In [108]:
dat_lists = [dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8]
file_id_start = 1
Path('./results/latent_features').mkdir(exist_ok = True)
for data in dat_lists:
    dat_lat = data_process.transform_features_with_NN(model, data[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
    dat_lat.to_csv(f"./results/latent_features/seq_{file_id_start}.csv", index=False, header=True)
    file_id_start += 1

## Save IDs

In [109]:
ids = pd.concat([dat1.PERS_ID, dat2.PERS_ID, dat3.PERS_ID, dat4.PERS_ID, dat5.PERS_ID, dat6.PERS_ID, dat7.PERS_ID, dat8.PERS_ID], axis=1, ignore_index=True)
ids.rename(columns={0:"seq1", 1:"seq2", 2:"seq3", 3:"seq4", 4:"seq5", 5:"seq6", 6:"seq7", 7:"seq8"}, inplace=True)
ids.fillna(-1, inplace=True)
ids = ids.astype(int)

In [110]:
ids

seq1     seq2     seq3     seq4     seq5     seq6     seq7     seq8
0     2778299  2778299  4095888  4095888  5066702  5066702  2786032  2786032
1     4033552  4033552  2774711  2774711  2778176  2778176  5066702  5066702
2     2778384  2778384  4393268  4393268  5071256  5071256  4164512  4164512
3     4122495  4122495  2773392  2773392  4069664  4069664  2778176  2778176
4     2773286  2773286  2772303  2772303  3338592  3338592  2778176  2778176
...       ...      ...      ...      ...      ...      ...      ...      ...
1955       -1  2849649       -1  2848538       -1  2867957       -1  4176793
1956       -1  2849524       -1  2852547       -1  4217656       -1  4057429
1957       -1  2845746       -1  2848563       -1  2866680       -1  3800329
1958       -1  2855862       -1  2860060       -1  2861097       -1  4098859
1959       -1  2852977       -1  2847836       -1  5029633       -1  5017248

[1960 rows x 8 columns]

In [111]:
Path('./results/ids').mkdir(exist_ok = True)
ids.to_csv("./results/ids/ids_seq_1_8.csv", index=True, header=True)

## Ratios with/without Std

In [112]:
disease_cols = [cols for cols in treated.columns if "CC" in cols and cols!='CC_DX_CNT']
other_binary_cols = ['MBR_GNDR', 'LOB', 'PREGNANCY', 'IP_BIN', 'ER_BIN', 'ACUTE']
binary_cols = other_binary_cols + disease_cols 
restricted_cols = ['PERS_ID', 'MYR', 'MEDICAID_LOB']
continuous_cols = [cols for cols in treated.columns if cols not in binary_cols + restricted_cols]

In [113]:
def calculate_continuous_d(t, c, std):
    Xt = np.mean(t)
    Xc = np.mean(c)
    St = np.std(t)
    Sc = np.std(c)
    if std==1: 
        d = np.abs((Xt-Xc)/(np.sqrt((St**2 + Sc**2)/2)))
    else:
        d = np.abs((Xt-Xc))
    return d

In [114]:
def calculate_binary_d(t, c, std):
    Pt = np.mean(t)
    Pc = np.mean(c)
    if std==1:
        d = np.abs((Pt-Pc)/(np.sqrt((Pt*(1-Pt)+Pc*(1-Pc))/2)))
    else:
        d = np.abs(Pt-Pc)
    return d

### 1.1 Whole Population as controls

In [146]:
std = 0

In [147]:
d_metric_before_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controls[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controls[[col]].values, std=std)
        d_metric_before_all.loc[0,col] = d

In [148]:
d_metric_after_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[:4]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_all.loc[i,col] = d
            i+=1

In [149]:
d_metric_after_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.03             0.88     344.68     14.29    279.28     0.12          0.01               0.03    0.02       0.09                 0.47          0.00       0.10           0.03       0.02        0.33    0.00              0.00         0.01               0.00               0.17                  0.01     0.10                0.00           0.46                     0.10               0.00         0.05             0.09       3.55                   0.01         0.04             0.17         0.01               0.01              0.07            0.00                  0.00                    0.00                  0.00                    0.02                        0.02            0.00            0.00              0.03             0.00               0.01             0.02            0.00          0.01         0.03            0.01          0.00      0.09            0.00         0.01                0.01                   0.00                   0.00        0.09             0.00           0.01   
1      0.03             1.01     383.73     16.25    279.28     0.15          0.01               0.03    0.02       0.10                 0.57          0.05       0.13           0.03       0.02        0.36    0.00              0.00         0.01               0.00               0.20                  0.01     0.12                0.00           0.53                     0.10               0.01         0.10             0.12       4.46                   0.01         0.05             0.22         0.00               0.02              0.09            0.00                  0.00                    0.00                  0.00                    0.01                        0.02            0.00            0.00              0.03             0.00               0.05             0.10            0.01          0.01         0.06            0.01          0.00      0.12            0.00         0.02                0.02                   0.00                   0.00        0.02             0.00           0.00   
2      0.03             0.58    1051.12      4.04    359.34     0.12          0.01               0.03    0.02       0.02                 0.35          0.04       0.04           0.02       0.02        0.28    0.00              0.00         0.02               0.00               0.08                  0.00     0.08                0.00           0.36                     0.09               0.01         0.04             0.05       2.14                   0.01         0.04             0.11         0.00               0.02              0.04            0.00                  0.00                    0.00                  0.00                    0.01                        0.01            0.00            0.00              0.02             0.00               0.07             0.05            0.02          0.01         0.01            0.01          0.01      0.05            0.01         0.02                0.00                   0.01                   0.00        0.21             0.00           0.01   
3  

In [150]:
d_metric_before_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.09            11.50     118.23      7.31    109.38     0.18          0.00               0.02    0.01       0.09                 0.67          0.00       0.15           0.00       0.03        0.38    0.02              0.01         0.07               0.00               0.16                  0.00     0.09                0.00           0.59                     0.10               0.01         0.04             0.11       4.12                   0.01         0.05             0.23         0.03               0.04              0.06            0.00                  0.00                    0.00                  0.00                    0.02                        0.02            0.00            0.00              0.03             0.00               0.08             0.01            0.00          0.02         0.02            0.01          0.01      0.12            0.00         0.05                0.02                   0.00                   0.00        0.05             0.01           0.03   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.20         0.01                0.01     0.47              0.10                    0.14            0.02             0.00       0.01        0.50                0.01              0.00 0.77       0.00    0.05    0.04   0.09  3.12  4.08     6.09     0.73     0.73    1.26     0.19     0.22    0.34

In [151]:
ratio_1_4 = d_metric_after_all.values / d_metric_before_all.values
ratio_1_4 = pd.DataFrame(ratio_1_4, columns=d_metric_after_all.columns)

In [152]:
ratio_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.27             0.08       2.92      1.96      2.55     0.64          2.87               1.50    1.09       1.04                 0.70          0.16       0.67          11.28       0.60        0.87    0.23              0.35         0.18               4.67               1.06                  1.41     1.11                1.57           0.78                     0.94               0.67         1.15             0.79       0.86                   1.26         0.84             0.75         0.25               0.39              1.13            0.00                  0.00                    3.02                  0.00                    1.05                        1.00            0.00            0.00              0.90             0.00               0.17             1.61            1.35          0.93         1.59            0.94          0.00      0.75            1.02         0.19                0.34                   0.42                   0.00        1.92             0.07           0.16   
1      0.27             0.09       3.25      2.22      2.55     0.83          2.87               1.79    1.20       1.12                 0.85          9.75       0.86          13.96       0.77        0.94    0.23              0.13         0.16               2.34               1.26                  1.61     1.24                1.37           0.90                     0.96               1.09         2.18             1.07       1.08                   1.12         0.96             0.96         0.05               0.44              1.45            0.00                  0.00                    1.51                  0.00                    0.92                        1.00            0.00            0.00              0.94             0.00               0.55             9.30            5.68          0.99         2.55            0.90          0.14      0.96            0.25         0.47                0.98                   0.83                   1.52        0.42             0.35           0.01   
2      0.27             0.05       8.89      0.55      3.29     0.65          2.87               1.97    1.09       0.25                 0.52          8.97       0.30          10.39       0.67        0.74    0.13              0.13         0.30               4.67               0.50                  0.99     0.87                1.57           0.61                     0.84               0.95         0.99             0.48       0.52                   0.97         0.76             0.49         0.08               0.55              0.60            0.00                  5.70                    0.00                  0.00                    0.38                        0.51            0.00            0.00              0.57             0.00               0.85             4.14            6.98          0.65         0.47            0.80          1.73      0.38            1.53         0.49                0.08                   2.50                   0.00        4.25             0.07           0.41   
3  

In [153]:
mask_1_4 = ratio_1_4[ratio_1_4<=1.0]

In [154]:
mask_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.27             0.08        nan       nan       nan     0.64           nan                nan     nan        nan                 0.70          0.16       0.67            nan       0.60        0.87    0.23              0.35         0.18                nan                nan                   nan      nan                 nan           0.78                     0.94               0.67          nan             0.79       0.86                    nan         0.84             0.75         0.25               0.39               nan            0.00                  0.00                     nan                  0.00                     nan                         nan            0.00            0.00              0.90             0.00               0.17              nan             nan          0.93          nan            0.94          0.00      0.75             nan         0.19                0.34                   0.42                   0.00         nan             0.07           0.16   
1      0.27             0.09        nan       nan       nan     0.83           nan                nan     nan        nan                 0.85           nan       0.86            nan       0.77        0.94    0.23              0.13         0.16                nan                nan                   nan      nan                 nan           0.90                     0.96                nan          nan              nan        nan                    nan         0.96             0.96         0.05               0.44               nan            0.00                  0.00                     nan                  0.00                    0.92                         nan            0.00            0.00              0.94             0.00               0.55              nan             nan          0.99          nan            0.90          0.14      0.96            0.25         0.47                0.98                   0.83                    nan        0.42             0.35           0.01   
2      0.27             0.05        nan      0.55       nan     0.65           nan                nan     nan       0.25                 0.52           nan       0.30            nan       0.67        0.74    0.13              0.13         0.30                nan               0.50                  0.99     0.87                 nan           0.61                     0.84               0.95         0.99             0.48       0.52                   0.97         0.76             0.49         0.08               0.55              0.60            0.00                   nan                    0.00                  0.00                    0.38                        0.51            0.00            0.00              0.57             0.00               0.85              nan             nan          0.65         0.47            0.80           nan      0.38             nan         0.49                0.08                    nan                   0.00         nan             0.07           0.41   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [155]:
mask_1_4.columns[~mask_1_4.isnull().any()].size

In [156]:
mask_1_4.columns[~mask_1_4.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_ADHD', 'CC_ANXIETY_DISORDER', 'CC_ASTHMA', 'CC_AUTISM', 'CC_BIPOLAR', 'CC_BPH', 'CC_BREAST_CANCER', 'CC_CATARACT', 'CC_DEPRESSION', 'CC_DEPRESSIVE_DISORDERS', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_GLAUCOMA', 'CC_HEARING_IMPAIR', 'CC_HEPATITIS_A', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_HEPATITIS_GEN', 'CC_HIP_FRACTURE', 'CC_HYPERLIPIDEMIA', 'CC_INT_DISAB', 'CC_LIVER', 'CC_MIGRAINE', 'CC_MOBILITY_IMPAIR', 'CC_OSTEOPOROSIS', 'CC_PERIPHERAL', 'CC_PERSONALITY', 'CC_PROSTATE_CANCER', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_TOBACCO', 'CC_TRAUMATIC_BRAIN', 'LOB', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'ER_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [157]:
mask_1_4.columns[mask_1_4.isnull().all()].size

In [158]:
mask_1_4.columns[mask_1_4.isnull().all()]

Index(['ALLOW_AMT', 'CC_ALZHEIMER', 'CC_ALZHEIMER_PLUS', 'CC_ATRIAL_FIB', 'CC_CEREBRAL_PALSY', 'CC_CYSTIC_FIBROSIS', 'CC_HYPOTHYROID', 'CC_SPINA_BIFIDA'], dtype='object')

### 1.2 Only Medicaid Controls

In [159]:
d_metric_before_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controlsM[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controlsM[[col]].values, std=std)
        d_metric_before_med.loc[0,col] = d

In [160]:
d_metric_before_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.05             2.89      37.35      4.51    143.82     0.14          0.01               0.03    0.02       0.08                 0.58          0.05       0.12           0.03       0.02        0.33    0.00              0.00         0.02               0.00               0.19                  0.01     0.10                0.00           0.51                     0.09               0.00         0.08             0.10       4.32                   0.01         0.04             0.22         0.00               0.02              0.08            0.00                  0.00                    0.00                  0.00                    0.01                        0.02            0.00            0.00              0.02             0.00               0.05             0.12            0.03          0.01         0.05            0.01          0.00      0.12            0.00         0.03                0.02                   0.00                   0.00        0.06             0.00           0.01   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.18         0.01                0.00     0.42              0.09                    0.12            0.02             0.00       0.01        0.40                0.01              0.00 0.00       0.00    0.05    0.02   0.06  3.03  3.26     5.23     0.70     0.60    1.12     0.18     0.17    0.29

In [161]:
d_metric_after_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[4:]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_med.loc[i,col] = d
            i+=1

In [162]:
d_metric_after_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.03             0.23     688.90     22.19    279.28     0.15          0.01               0.03    0.01       0.08                 0.48          0.04       0.11           0.03       0.02        0.28    0.00              0.01         0.01               0.00               0.20                  0.00     0.11                0.01           0.44                     0.09               0.01         0.10             0.11       3.87                   0.01         0.04             0.20         0.00               0.01              0.10            0.00                  0.00                    0.00                  0.00                    0.01                        0.02            0.00            0.00              0.02             0.00               0.04             0.10            0.03          0.01         0.04            0.02          0.00      0.10            0.00         0.01                0.02                   0.00                   0.00        0.05             0.01           0.01   
1      0.03             0.08     683.77     22.01    279.28     0.14          0.01               0.03    0.01       0.08                 0.51          0.05       0.12           0.03       0.02        0.30    0.00              0.01         0.01               0.00               0.20                  0.00     0.11                0.01           0.46                     0.09               0.01         0.09             0.11       4.05                   0.01         0.04             0.22         0.01               0.01              0.09            0.00                  0.00                    0.00                  0.00                    0.01                        0.02            0.00            0.00              0.02             0.00               0.05             0.10            0.02          0.01         0.05            0.01          0.00      0.12            0.00         0.02                0.02                   0.00                   0.00        0.03             0.01           0.00   
2      0.03             0.29     693.64     22.19    279.28     0.15          0.01               0.03    0.01       0.11                 0.52          0.06       0.12           0.03       0.03        0.33    0.00              0.00         0.00               0.00               0.22                  0.01     0.11                0.01           0.50                     0.10               0.00         0.11             0.13       4.35                   0.01         0.05             0.23         0.01               0.01              0.09            0.00                  0.00                    0.00                  0.00                    0.02                        0.02            0.00            0.00              0.03             0.00               0.04             0.10            0.02          0.01         0.07            0.01          0.00      0.12            0.00         0.04                0.02                   0.00                   0.00        0.05             0.00           0.00   
3  

In [163]:
ratio_5_8 = d_metric_after_med.values / d_metric_before_med.values
ratio_5_8 = pd.DataFrame(ratio_5_8, columns=d_metric_after_med.columns)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [164]:
ratio_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.52             0.08      18.45      4.92      1.94     1.09          1.09               1.06    0.91       0.98                 0.83          0.70       0.94           1.25       1.12        0.85    6.21              2.43         0.42               1.44               1.08                  0.66     1.14                3.62           0.88                     1.00               1.93         1.23             1.10       0.90                   1.07         0.91             0.90         1.01               0.49              1.16           12.03                  0.00                    1.60                  0.00                    1.13                        0.80            0.00            0.00              0.76             2.71               0.87             0.86            1.27          0.81         0.87            1.70          0.57      0.83            0.00         0.36                0.99                   1.00                   0.00        0.86             1.67           1.39   
1      0.52             0.03      18.31      4.88      1.94     1.04          1.03               0.99    0.84       0.92                 0.88          0.98       1.06           1.07       1.26        0.92    5.57              2.02         0.74               0.72               1.07                  0.66     1.12                1.81           0.91                     0.97               1.65         1.12             1.16       0.94                   1.07         0.95             0.99         1.12               0.65              1.06            6.01                  0.00                    1.20                  0.00                    1.08                        0.96            0.00            0.00              0.92             0.68               1.05             0.85            0.88          0.81         1.07            1.59          0.29      0.94            0.00         0.48                1.04                   0.75                   0.00        0.54             1.67           0.33   
2      0.52             0.10      18.57      4.92      1.94     1.12          1.09               1.06    0.91       1.27                 0.90          1.15       1.05           1.25       1.55        1.02    6.21              1.33         0.21               0.00               1.13                  0.93     1.14                3.62           0.99                     1.06               1.36         1.41             1.33       1.01                   1.07         1.11             1.07         2.36               0.49              1.14            0.00                  0.00                    0.00                  0.00                    1.52                        1.18            0.00            0.00              1.38             0.00               0.79             0.83            0.88          1.02         1.32            1.27          0.57      0.96            0.00         1.19                1.09                   0.00                   4.43        0.86             0.69           0.15   
3  

In [165]:
mask_5_8 = ratio_5_8[ratio_5_8<=1.0]

In [166]:
mask_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.52             0.08        nan       nan       nan      nan           nan                nan    0.91       0.98                 0.83          0.70       0.94            nan        nan        0.85     nan               nan         0.42                nan                nan                  0.66      nan                 nan           0.88                     1.00                nan          nan              nan       0.90                    nan         0.91             0.90          nan               0.49               nan             nan                  0.00                     nan                  0.00                     nan                        0.80            0.00            0.00              0.76              nan               0.87             0.86             nan          0.81         0.87             nan          0.57      0.83            0.00         0.36                0.99                    nan                   0.00        0.86              nan            nan   
1      0.52             0.03        nan       nan       nan      nan           nan               0.99    0.84       0.92                 0.88          0.98        nan            nan        nan        0.92     nan               nan         0.74               0.72                nan                  0.66      nan                 nan           0.91                     0.97                nan          nan              nan       0.94                    nan         0.95             0.99          nan               0.65               nan             nan                  0.00                     nan                  0.00                     nan                        0.96            0.00            0.00              0.92             0.68                nan             0.85            0.88          0.81          nan             nan          0.29      0.94            0.00         0.48                 nan                   0.75                   0.00        0.54              nan           0.33   
2      0.52             0.10        nan       nan       nan      nan           nan                nan    0.91        nan                 0.90           nan        nan            nan        nan         nan     nan               nan         0.21               0.00                nan                  0.93      nan                 nan           0.99                      nan                nan          nan              nan        nan                    nan          nan              nan          nan               0.49               nan            0.00                  0.00                    0.00                  0.00                     nan                         nan            0.00            0.00               nan             0.00               0.79             0.83            0.88           nan          nan             nan          0.57      0.96            0.00          nan                 nan                   0.00                    nan        0.86             0.69           0.15   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [167]:
mask_5_8.columns[~mask_5_8.isnull().any()].size

In [168]:
mask_5_8.columns[~mask_5_8.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_ANXIETY_DISORDER', 'CC_CATARACT', 'CC_COLORECTAL_CANCER', 'CC_HEARING_IMPAIR', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_LUNG_CANCER', 'CC_OBESITY', 'CC_PERSONALITY', 'CC_PRESSURE', 'CC_PROSTATE_CANCER', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_SPINA_BIFIDA', 'CC_TOBACCO', 'PREGNANCY'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [169]:
mask_5_8.columns[mask_5_8.isnull().all()].size

In [170]:
mask_5_8.columns[mask_5_8.isnull().all()]

Index(['ALLOW_AMT', 'ALLOW_ER', 'ALLOW_IP', 'CC_ADHD', 'CC_ALZHEIMER', 'CC_ATRIAL_FIB', 'CC_AUTISM', 'CC_BPH', 'CC_BREAST_CANCER', 'CC_CHRONIC_KIDNEY', 'CC_COPD', 'CC_CYSTIC_FIBROSIS', 'CC_DEVELOP_DELAYS', 'CC_DIABETES', 'CC_DISAB_DX_CNT', 'CC_ENDOMETRIAL_CANCER', 'CC_GLAUCOMA', 'CC_HEART_FAILURE', 'CC_HEPATITIS_C_CHRONIC', 'CC_LEARN_DISAB', 'CC_STROKE', 'CC_TRAUMATIC_BRAIN', 'LOB', 'IP_BIN', 'ER_BIN', 'ACUTE', 'IP_BIN2', 'ER_BIN2', 'ACUTE2'], dtype='object')

### 1.3 Merge and Save ratio

In [171]:
final_ratio = pd.concat([ratio_1_4, ratio_5_8], axis=0, ignore_index=True)
final_ratio

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.27             0.08       2.92      1.96      2.55     0.64          2.87               1.50    1.09       1.04                 0.70          0.16       0.67          11.28       0.60        0.87    0.23              0.35         0.18               4.67               1.06                  1.41     1.11                1.57           0.78                     0.94               0.67         1.15             0.79       0.86                   1.26         0.84             0.75         0.25               0.39              1.13            0.00                  0.00                    3.02                  0.00                    1.05                        1.00            0.00            0.00              0.90             0.00               0.17             1.61            1.35          0.93         1.59            0.94          0.00      0.75            1.02         0.19                0.34                   0.42                   0.00        1.92             0.07           0.16   
1      0.27             0.09       3.25      2.22      2.55     0.83          2.87               1.79    1.20       1.12                 0.85          9.75       0.86          13.96       0.77        0.94    0.23              0.13         0.16               2.34               1.26                  1.61     1.24                1.37           0.90                     0.96               1.09         2.18             1.07       1.08                   1.12         0.96             0.96         0.05               0.44              1.45            0.00                  0.00                    1.51                  0.00                    0.92                        1.00            0.00            0.00              0.94             0.00               0.55             9.30            5.68          0.99         2.55            0.90          0.14      0.96            0.25         0.47                0.98                   0.83                   1.52        0.42             0.35           0.01   
2      0.27             0.05       8.89      0.55      3.29     0.65          2.87               1.97    1.09       0.25                 0.52          8.97       0.30          10.39       0.67        0.74    0.13              0.13         0.30               4.67               0.50                  0.99     0.87                1.57           0.61                     0.84               0.95         0.99             0.48       0.52                   0.97         0.76             0.49         0.08               0.55              0.60            0.00                  5.70                    0.00                  0.00                    0.38                        0.51            0.00            0.00              0.57             0.00               0.85             4.14            6.98          0.65         0.47            0.80          1.73      0.38            1.53         0.49                0.08                   2.50                   0.00        4.25             0.07           0.41   
3  

In [172]:
Path('./results/ratio').mkdir(exist_ok = True)
final_ratio.to_csv("./results/ratio/ratio_without_std_1_8.csv", index=False, header=True)